In [ ]:
import pandas as pd
import numpy as np
from plotnine import *
from tensorflow import keras
from keras import models
from keras import layers
from keras import losses
from tensorflow.keras import optimizers
from keras import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression # Logistic Regression Model
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix

from sklearn.model_selection import KFold # k-fold cv
from sklearn.model_selection import LeaveOneOut #LOO cv
from sklearn.model_selection import cross_val_score # cross validation metrics
from sklearn.model_selection import cross_val_predict # cross validation metrics

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.metrics import mean_squared_error, r2_score #model evaluation



In [ ]:
db=pd.read_csv("games.csv")
#db.head()


db["winner"] = db["winner"].astype('category')
db['winner_code']=db["winner"].cat.codes

db["opening_eco"]=db["opening_eco"].astype('category')
db["opening_eco_code"]=db["opening_eco"].cat.codes

db["opening_name"]=db["opening_name"].astype('category')
db["opening_name_code"]=db["opening_name"].cat.codes

db["victory_status"]=db["victory_status"].astype('category')
db["victory_status_code"]=db["victory_status"].cat.codes

db["increment_code"]=db["increment_code"].astype('category')
db["increment_code_num"]=db["increment_code"].cat.codes

In [ ]:
#Setting Categorical Codes for Variables

In [ ]:
ELOconditionsW = [
    (db['white_rating'] >= 0) & (db['white_rating'] <= 1450),
    (db['white_rating'] >= 1451) & (db['white_rating'] <= 1700),
    (db['white_rating'] >= 1701)
    ]

ELOconditionsB = [
    (db['black_rating'] >= 0) & (db['black_rating'] <= 1450),
    (db['black_rating'] >= 1451) & (db['black_rating'] <= 1700),
    (db['black_rating'] >= 1701)
    ]

values = ['0', '1', '2']

db['tierW'] = np.select(ELOconditionsW, values)
db['tierB'] = np.select(ELOconditionsB, values)

db['tierW'] = db['tierW'].astype(str).astype('category')
db['tierWcode'] = db['tierW'].cat.codes
db['tierB'] = db['tierB'].astype(str).astype('category')
db['tierBcode'] = db['tierB'].cat.codes




#db[['black_rating','white_rating','tierB','tierW','winner']].iloc[0:25]

counter =0
for i in db['tierW']:
    if i == '2':
        counter +=1
        
print(counter)

In [ ]:
#setting skill boundaries

In [ ]:
db.head()

In [ ]:
predictors=["opening_eco_code","turns"]

X_train, X_test, y_train, y_test = train_test_split(db[predictors], db["tierWcode"], test_size=0.2)


zscore = StandardScaler()
zscore.fit(X_train)

Xz_train = zscore.transform(X_train)
Xz_test = zscore.transform(X_test)

myLogit = LogisticRegression(penalty = "none") # create

myLogit.fit(Xz_train, y_train) #fit

predictedVals = myLogit.predict(Xz_test)# predict

In [ ]:
accuracy_score(y_test,predictedVals)

In [ ]:
plot_confusion_matrix(myLogit, Xz_test, y_test)

In [ ]:
#Logistic Regression
#Input - openings and turns
#output - tier of white player
#guessing accuracy

In [ ]:
predictors=["white_rating","opening_eco_code","black_rating","rated"]
X = db[predictors]
Y = db["turns"]

In [ ]:
zScore = StandardScaler()
zScore.fit(X)

Xz = zScore.transform(X)

In [ ]:
LR_Model1 = LinearRegression()

In [ ]:
LR_Model1.fit(Xz, Y)

In [ ]:
turn_pred = LR_Model1.predict(Xz)

turn_pred[1:10]

In [ ]:
mean_squared_error(Y, price_pred)

In [ ]:
r2_score(Y, price_pred)

In [ ]:
assump = pd.DataFrame({"error":Y - turn_pred, "predicted": turn_pred})

(ggplot(assump, aes(x = "predicted", y = "error")) + geom_point() + theme_minimal())

In [ ]:
coefficients = pd.DataFrame({"Coef":LR_Model1.coef_,
              "Name": predictors})
coefficients = coefficients.append({"Coef": LR_Model1.intercept_,
                    "Name": "intercept"}, ignore_index = True)

coefficients

In [ ]:
#Linear Regression
#Input - white rating, black rating, openings, and whether match was rated
#output - turns
#guessing accuracy, but strong correlation with player ratings and whether match was rated

In [ ]:
#Next Linear Reg

In [ ]:
predictors=["turns","opening_eco_code","black_rating"]
X = db[predictors]
Y = db["white_rating"]

In [ ]:
zScore = StandardScaler()
zScore.fit(X)

Xz = zScore.transform(X)

In [ ]:
LR_Model2 = LinearRegression()

In [ ]:
LR_Model2.fit(Xz, Y)

In [ ]:
elo_pred = LR_Model2.predict(Xz)

elo_pred[1:10]

In [ ]:
mean_squared_error(Y, elo_pred)

In [ ]:
r2_score(Y, elo_pred)

In [ ]:
assump = pd.DataFrame({"error":Y - elo_pred, "predicted": elo_pred})

(ggplot(assump, aes(x = "predicted", y = "error")) + geom_point() + theme_minimal())

In [ ]:
coefficients = pd.DataFrame({"Coef":LR_Model2.coef_,
              "Name": predictors})
coefficients = coefficients.append({"Coef": LR_Model2.intercept_,
                    "Name": "intercept"}, ignore_index = True)

coefficients

In [ ]:
#Linear Regression
#Input - black rating, openings, and turns
#output - white rating
#guessing accuracy, but strong correlation black rating

In [ ]:
'''
db.head(50)
counter =0
for i in db['tierW']:
    if i == '3':
        counter +=1
        
print(counter)


db["winner"] = db["winner"].astype('category')
db['winner_code']=db["winner"].cat.codes

db["opening_eco"]=db["opening_eco"].astype('category')
db["opening_eco_code"]=db["opening_eco"].cat.codes

db["opening_name"]=db["opening_name"].astype('category')
db["opening_name_code"]=db["opening_name"].cat.codes

db["victory_status"]=db["victory_status"].astype('category')
db["victory_status_code"]=db["victory_status"].cat.codes

db["increment_code"]=db["increment_code"].astype('category')
db["increment_code_num"]=db["increment_code"].cat.codes


data.speeding.plot.density(color='green')
plt.title('Density plot for Speeding')
plt.show()


#X=db[predictors].values.to_list()

#y=db["tierW"].values.to_list()

#db = pd.DataFrame(db)

#db = np.asarray(db["tierW"]).astype('float32')
#db = np.asarray(db["tierB"]).astype('float32')


predictors=["opening_eco_code","turns"]
#predictors=["turns", "opening_ply"]
db.shape
db.dtypes


x_train, x_test, y_train, y_test =train_test_split(db[predictors], db["tierWcode"], test_size=.2)
x_train= x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

len(x_train)

x_val=x_train[:1500]
partial_x_train = x_train[1500:]
y_val=y_train[:1500]
partial_y_train = y_train[1500: ]

model=models.Sequential()
model.add(layers.Dense(4, activation = 'relu', input_shape=(2,)))
model.add(layers.Dense(5, activation = 'relu'))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dense(1, activation= 'sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#categorical cross entropy
#output layer give threee layers
#sigmoid to softmax


[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=200,
                    batch_size=512,
                    validation_data=(x_val, y_val))
results=model.evaluate(x_test, y_test)
results

'''

In [ ]:
#FFNN
#Input - openings and turns
#output - white tier
#guessing accuracy